In [1]:
import numpy as np
import scipy.linalg as sla
import numpy.linalg as la
from scipy.special import expit as sigmoid
import scipy as sc
from statsmodels.tsa.api import VAR

In [2]:
varables = 3
observations = 1400
window = 2
#to use later 
times = 5

In [3]:
def qDag(w):
    return np.trace(sc.linalg.expm(np.multiply(w,w))) - w.shape[0]
def get_r(off):
    return lambda n: np.random.rand(n)+off

In [4]:
dagY = sc.sparse.random(varables, varables*window, density=0.15, data_rvs=get_r(.5)).A

dagX = np.eye(varables, varables)
while qDag(dagX) != 0:
    dagX = sc.sparse.random(varables, varables, density=0.25, data_rvs=get_r(.5)).A
    np.fill_diagonal(dagX, 0)

In [5]:
print(qDag(dagX))
print(dagX)
print(dagY)

0.0
[[0.         0.         0.60279788]
 [0.         0.         0.        ]
 [0.         1.09403295 0.        ]]
[[0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.63979639 0.         1.07154646]
 [0.         0.         0.         0.         1.48169739 0.        ]]


In [6]:
def rand(n):
    return 2*(np.random.rand(n)-.5)

Y_pre = rand(observations*varables*window).reshape(varables*window, observations)
print(dagY.shape)
print(Y_pre.shape)

(3, 6)
(6, 1400)


In [7]:
#X = XW + YA + Z

def randFlat(X):
    return rand(X.shape[0]*X.shape[1]).reshape(*X.shape)

def onesFlat(X):
    return np.ones(X.shape[0]*X.shape[1]).reshape(*X.shape)

def update(preY, preX, dagY, dagX, rand):
    Xy = np.matmul(dagY, preY)
    Xr = rand(preX)
    Xx = np.matmul(dagX, Xr)
    X = Xr + Xx + Xy
    
    return X

def step(preY, dagY, dagX, window, var, rand):
    preX = preY[:var]
    preY = preY[:window*var]
    return update(preY, preX, dagY, dagX, rand)

def makeSynthData(preY, times, obs, var, pad, rand=rand):
    window = preY.shape[0]//var
    
    for i in range(pad+window):
        newX = step(preY, dagY, dagX, window, var, rand)
        preY = np.append(newX, preY, axis=0)

    Y = preY[:window*var]

    for i in range(times - window):
        newX = step(Y, dagY, dagX, window, var, rand)
        Y = np.append(newX, Y, axis=0)
        
    return Y
    



#later
#W = np.random.rand(varables*observations).reshape(varables,observations)

In [8]:
Y_pre = np.ones(observations*varables*window).reshape(varables*window, observations)
XY = makeSynthData(Y_pre, times, observations, varables, 0, randFlat)


In [9]:
Y=XY[varables:window*varables+varables]
X=XY[:varables]

In [10]:
XY

array([[ 0.84451187, -1.07956641, -0.59938871, ..., -0.03042771,
        -0.11486132, -0.55207551],
       [ 3.10682466,  2.4738767 ,  5.84112258, ...,  5.92889012,
         2.89329739,  4.82253118],
       [ 3.90240578,  0.96240296,  1.57326366, ...,  4.45089193,
         3.87003019,  2.19973137],
       ...,
       [ 0.19557798,  0.52662838,  0.00739046, ...,  0.92649488,
         0.55386699,  0.38903255],
       [ 2.34018761,  1.70668865,  2.52804347, ...,  2.25652068,
         2.71011964,  1.79740467],
       [ 2.0146649 ,  0.81424269,  1.44679534, ...,  2.50126002,
         3.18556485,  1.32630718]])

In [11]:
C_init = np.random.rand(varables*varables*window + varables*varables)

In [12]:
def f_DYNOTEARS(X, Y, lA, lB):
    def aux(C):
        a = C[:varables*window*varables].reshape(varables, varables*window)
        w = C[varables*window*varables:].reshape(varables, varables)
    
        loss = np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/np.prod(X.shape)
        #print(loss)
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

In [13]:
def constr_DYNOTEARS(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS(X,Y,.001,.1) , C_init, constraints=cons).x

In [14]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [15]:
A.round(1)

array([[-0. ,  0. , -0. ,  0. , -0. , -0. ],
       [ 0. ,  0. , -0. ,  0.6, -0. ,  1.1],
       [ 0. ,  0. , -0. , -0.5,  1.4, -0.7]])

In [16]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.6, 0. , 1.1],
       [0. , 0. , 0. , 0. , 1.5, 0. ]])

In [17]:
W.round(1)

array([[ 0. ,  0. ,  0. ],
       [-0. ,  0. ,  0. ],
       [ 0.1,  0.7,  0. ]])

In [18]:
dagX.round(1)

array([[0. , 0. , 0.6],
       [0. , 0. , 0. ],
       [0. , 1.1, 0. ]])

In [19]:
dagX.T.round(1)

array([[0. , 0. , 0. ],
       [0. , 0. , 1.1],
       [0.6, 0. , 0. ]])

In [20]:
def H1ldet(W):
    det = np.linalg.det(np.eye(W.shape[0]) - np.multiply(W,W))
    if (det <= 0):
        print(det)
    return -np.log(det)

def constr_DAGMA(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return H1ldet(w)

A_init = np.random.rand(varables*varables*window)

W_init = np.random.rand(varables*varables).reshape(varables, varables)
np.fill_diagonal(W_init, 0)
while H1ldet(W_init) is np.nan:
    W_init = np.random.rand(varables*varables).reshape(varables, varables)
    np.fill_diagonal(W, 0)

C_init = np.append(A_init, W_init.reshape(varables*varables), axis = 0)

cons = {'type':'eq', 'fun': constr_DAGMA}

#C = sc.optimize.minimize(f_DYNOTEARS(X,Y,.01,.1) , C_init, constraints=cons).x

In [21]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [22]:
A[A<.3] = 0
A.round()

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 1., 0.]])

In [23]:
dagY.round()

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 1., 0.]])

In [24]:
W.round(1)

array([[ 0. ,  0. ,  0. ],
       [-0. ,  0. ,  0. ],
       [ 0.1,  0.7,  0. ]])

In [25]:
dagX.round(1)

array([[0. , 0. , 0.6],
       [0. , 0. , 0. ],
       [0. , 1.1, 0. ]])

In [26]:
dagX.T.round(1)

array([[0. , 0. , 0. ],
       [0. , 0. , 1.1],
       [0.6, 0. , 0. ]])

In [27]:
C_init = np.random.rand(varables*varables*window + varables*varables)
H1ldet(C_init.T)

-7.684705686937305


/var/folders/z_/p0tz320d6j13q0rkv_f35brm0000gn/T/ipykernel_18744/3164180601.py:5: RuntimeWarning: invalid value encountered in log
  return -np.log(det)


nan

In [28]:
def metrics(A,dagY, W, dagX):
    A = np.absolute(A) > 0.3
    dagY = np.absolute(dagY) > .5
    W = np.absolute(W) > 0.3
    dagX = np.absolute(dagX) > .5
    
    preWrong = A != dagY
    wWrong = W != dagX
    #assumes cant be both wrong and inv
    wInverse = W.T & dagX
    
    if ((W + W.T)==2).any():
        print(W)
        print(W.T)
        raise "error"
    
    #print(A)
    #print(dagY)
    #print(W)
    #print(dagX)
    #print()
    #print(preWrong)
    #print(wWrong)
    #print(wInverse)
    
    return {
        "preWrong": sum(sum(preWrong)),
        "wWrong": sum(sum(wWrong)),
        "wInverse": sum(sum(wInverse))
    }
    

In [29]:
metrics(A,dagY, W, dagX)

{'preWrong': 0, 'wWrong': 1, 'wInverse': 0}

In [30]:
A.round()

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 1., 0.]])

In [31]:
dagY.round()

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 1., 0.]])

In [32]:
np.matmul(np.eye(varables, varables) - W, X).round(1)

array([[ 0.8, -1.1, -0.6, ..., -0. , -0.1, -0.6],
       [ 3.1,  2.5,  5.8, ...,  5.9,  2.9,  4.8],
       [ 1.7, -0.6, -2.3, ...,  0.5,  1.9, -1. ]])

In [33]:
def transformTimeData(data, window, times):
    variables = data.shape[0]//times
    print(data.shape[0]/times)
    
    snapshots = times - window - 1
    
    ret = []
    
    for i in range(snapshots):
        slic = data[i*variables:i*variables+(window+1)*variables]
        ret.append( slic)
    
    return np.hstack(ret)
    
    

In [34]:
XY_stack = transformTimeData(XY, window, times)

3.0


In [35]:
Y_stack=XY_stack[varables:window*varables+varables]
X_stack=XY_stack[:varables]


In [36]:
print(XY.shape)
print(XY_stack.shape)
print(Y_stack.shape)
print(X_stack.shape)

(15, 1400)
(9, 2800)
(6, 2800)
(3, 2800)


In [37]:
XY.shape

(15, 1400)

In [38]:
XY_stack

array([[ 0.84451187, -1.07956641, -0.59938871, ...,  0.44023687,
         0.22243026,  0.52355365],
       [ 3.10682466,  2.4738767 ,  5.84112258, ...,  0.41082145,
         0.80628532,  0.10845134],
       [ 3.90240578,  0.96240296,  1.57326366, ...,  2.35784659,
         2.90278807,  1.70489404],
       ...,
       [-0.80792361, -0.29018851,  1.33792527, ..., -0.26114815,
        -1.16527905, -0.07501112],
       [ 2.77903565,  1.15421834,  0.66760176, ...,  1.46296122,
         2.4517169 ,  1.48250227],
       [ 3.63267786,  2.81572036,  3.75929592, ...,  0.30860306,
         1.67148319,  0.29349959]])

In [39]:
X_stack

array([[ 0.84451187, -1.07956641, -0.59938871, ...,  0.44023687,
         0.22243026,  0.52355365],
       [ 3.10682466,  2.4738767 ,  5.84112258, ...,  0.41082145,
         0.80628532,  0.10845134],
       [ 3.90240578,  0.96240296,  1.57326366, ...,  2.35784659,
         2.90278807,  1.70489404]])

In [40]:
Y_stack

array([[-0.15641718, -0.07613062, -0.55656899, ...,  1.52782078,
        -0.23897312,  1.12553678],
       [ 4.31152743,  1.39225503, -0.6106612 , ...,  2.93902861,
         3.27889274,  1.60016699],
       [ 4.70204967,  3.41669166,  0.99215385, ...,  3.88350732,
         3.67027602,  3.52401136],
       [-0.80792361, -0.29018851,  1.33792527, ..., -0.26114815,
        -1.16527905, -0.07501112],
       [ 2.77903565,  1.15421834,  0.66760176, ...,  1.46296122,
         2.4517169 ,  1.48250227],
       [ 3.63267786,  2.81572036,  3.75929592, ...,  0.30860306,
         1.67148319,  0.29349959]])

In [41]:
cons = {'type':'eq', 'fun': constr_DYNOTEARS}
C = sc.optimize.minimize(f_DYNOTEARS(X_stack,Y_stack,.01,.1) , C_init, constraints=cons).x

In [42]:
A = C[:varables*window*varables].reshape(varables, varables*window)
W = C[varables*window*varables:].reshape(varables, varables)
np.fill_diagonal(W, 0)

In [43]:
abs(A.round(1))

array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.6, 0. , 1.1],
       [0. , 0. , 0. , 0.4, 1.4, 0.6]])

In [44]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.6, 0. , 1.1],
       [0. , 0. , 0. , 0. , 1.5, 0. ]])

In [45]:
W.round(2)

array([[ 0.  , -0.  ,  0.  ],
       [-0.  ,  0.  ,  0.  ],
       [ 0.08,  0.6 ,  0.  ]])

In [46]:
dagX.round(1)

array([[0. , 0. , 0.6],
       [0. , 0. , 0. ],
       [0. , 1.1, 0. ]])

In [47]:
dagX.T.round(1)

array([[0. , 0. , 0. ],
       [0. , 0. , 1.1],
       [0.6, 0. , 0. ]])

In [48]:
#tensor representation for timeseries
dagYtesnor = np.transpose(np.array([dagY[:,i*varables:(i+1)*varables] for i in range(window)]),(0,2,1))

dagXtesnor = dagX
print(dagYtesnor.round().shape)

XYtesnor = XY.reshape(times, varables, observations)
print(XYtesnor.shape)

(2, 3, 3)
(5, 3, 1400)


In [49]:
XY_stack

array([[ 0.84451187, -1.07956641, -0.59938871, ...,  0.44023687,
         0.22243026,  0.52355365],
       [ 3.10682466,  2.4738767 ,  5.84112258, ...,  0.41082145,
         0.80628532,  0.10845134],
       [ 3.90240578,  0.96240296,  1.57326366, ...,  2.35784659,
         2.90278807,  1.70489404],
       ...,
       [-0.80792361, -0.29018851,  1.33792527, ..., -0.26114815,
        -1.16527905, -0.07501112],
       [ 2.77903565,  1.15421834,  0.66760176, ...,  1.46296122,
         2.4517169 ,  1.48250227],
       [ 3.63267786,  2.81572036,  3.75929592, ...,  0.30860306,
         1.67148319,  0.29349959]])

In [50]:
Y_stack=XY_stack[varables:window*varables+varables]
X_stack=XY_stack[:varables]

In [51]:
Y_stack

array([[-0.15641718, -0.07613062, -0.55656899, ...,  1.52782078,
        -0.23897312,  1.12553678],
       [ 4.31152743,  1.39225503, -0.6106612 , ...,  2.93902861,
         3.27889274,  1.60016699],
       [ 4.70204967,  3.41669166,  0.99215385, ...,  3.88350732,
         3.67027602,  3.52401136],
       [-0.80792361, -0.29018851,  1.33792527, ..., -0.26114815,
        -1.16527905, -0.07501112],
       [ 2.77903565,  1.15421834,  0.66760176, ...,  1.46296122,
         2.4517169 ,  1.48250227],
       [ 3.63267786,  2.81572036,  3.75929592, ...,  0.30860306,
         1.67148319,  0.29349959]])

In [52]:
X_stack

array([[ 0.84451187, -1.07956641, -0.59938871, ...,  0.44023687,
         0.22243026,  0.52355365],
       [ 3.10682466,  2.4738767 ,  5.84112258, ...,  0.41082145,
         0.80628532,  0.10845134],
       [ 3.90240578,  0.96240296,  1.57326366, ...,  2.35784659,
         2.90278807,  1.70489404]])

In [53]:
def diff(X, w, a):
    X_check = X[:X.shape[0]-a.shape[0]]
    
    Xw = np.matmul(w, X_check)
    #print(a.shape)
    #print(X.shape[0]-window+1)
    Xa = np.array([np.tensordot(a, X[i:i+window], axes=((0,1),(0,1)))
                    for i in range(1, X.shape[0]-window+1)])
    #print(X_check.shape)
    #print(Xw.shape)
    #print(Xa.shape)
    
    return X_check - Xw - Xa

def f_DYNOTEARS_TENSOR(X, lA, lB):
    def aux(C):
        a = C[:varables*window*varables].reshape(window, varables, varables)
        w = C[varables*window*varables:].reshape(varables, varables)
    
        loss = np.square(diff(X, w, a)).sum().sum().sum()/np.prod(X.shape)
        #print(loss)
        return loss + lB*np.linalg.norm(w.flatten(), 1) + lA*np.linalg.norm(a.flatten(), 1) 
    return aux

def ll(X, Y, w, a):
    return np.square(
                np.linalg.norm(
                    np.matmul(np.eye(varables, varables) - w, X)
                    - np.matmul(a, Y)
                    , "fro"
                )
            )/np.prod(X.shape)

In [54]:
Y_stack=XY_stack[varables:window*varables+varables]
X_stack=XY_stack[:varables]

In [55]:
def constr_DYNOTEARS(C):
    w = C[varables*window*varables:].reshape(varables, varables)
    return qDag(w)

cons = {'type':'eq', 'fun': constr_DYNOTEARS}

C = sc.optimize.minimize(f_DYNOTEARS_TENSOR(XYtesnor,.05,.05) , C_init, constraints=cons).x

In [56]:
A = C[:varables*window*varables].reshape(window, varables, varables)
W = C[varables*window*varables:].reshape(varables, varables)
print(ll(X_stack, Y_stack, dagX, dagY))

#print(ll(XYtesnor, W, A))

print(np.square(diff(XYtesnor, dagX, dagYtesnor)).sum().sum().sum()/np.prod(XYtesnor.shape))

print(np.square(diff(XYtesnor, W, A)).sum().sum().sum()/np.prod(XYtesnor.shape))

3.989690517440081
2.134650880793359
0.28736252797425266


In [57]:
A.shape

(2, 3, 3)

In [58]:
W.shape

(3, 3)

In [59]:
A.round(1)

array([[[-0. , -0. ,  0. ],
        [-0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ]],

       [[ 0. ,  0.4, -0. ],
        [-0. , -0. ,  1.2],
        [-0. ,  1.1, -0. ]]])

In [60]:
dagYtesnor.round(1)

array([[[0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0.6, 0. ],
        [0. , 0. , 1.5],
        [0. , 1.1, 0. ]]])

In [61]:
XYtesnor[1:1+window].round()

array([[[-0., -0., -1., ...,  0.,  0.,  1.],
        [ 4.,  1., -1., ...,  0.,  1.,  0.],
        [ 5.,  3.,  1., ...,  2.,  3.,  2.]],

       [[-1., -0.,  1., ...,  2., -0.,  1.],
        [ 3.,  1.,  1., ...,  3.,  3.,  2.],
        [ 4.,  3.,  4., ...,  4.,  4.,  4.]]])

In [62]:
dagYtesnor.round(1)

array([[[0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0.6, 0. ],
        [0. , 0. , 1.5],
        [0. , 1.1, 0. ]]])

In [63]:
np.tensordot(A, XYtesnor[1:1+window], axes=((0,1),(0,1))).round(1)

array([[-0. , -0. , -0. , ..., -0. , -0. , -0. ],
       [ 3.5,  2.9,  4.5, ...,  4.7,  3.8,  4.1],
       [ 3.4,  1.4,  0.8, ...,  3.6,  4. ,  2. ]])

In [64]:
XYtesnor[0].round()
XYtesnor[1:1+window].round()

array([[[-0., -0., -1., ...,  0.,  0.,  1.],
        [ 4.,  1., -1., ...,  0.,  1.,  0.],
        [ 5.,  3.,  1., ...,  2.,  3.,  2.]],

       [[-1., -0.,  1., ...,  2., -0.,  1.],
        [ 3.,  1.,  1., ...,  3.,  3.,  2.],
        [ 4.,  3.,  4., ...,  4.,  4.,  4.]]])

In [65]:
#np.array([dagY[:,i*varables:(i+1)*varables] for i in range(window)]).round(1)

In [66]:
dagY.round(1)

array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.6, 0. , 1.1],
       [0. , 0. , 0. , 0. , 1.5, 0. ]])

In [67]:
Y_stack.round()

array([[-0., -0., -1., ...,  2., -0.,  1.],
       [ 4.,  1., -1., ...,  3.,  3.,  2.],
       [ 5.,  3.,  1., ...,  4.,  4.,  4.],
       [-1., -0.,  1., ..., -0., -1., -0.],
       [ 3.,  1.,  1., ...,  1.,  2.,  1.],
       [ 4.,  3.,  4., ...,  0.,  2.,  0.]])

In [68]:
np.matmul(dagY, Y_stack).round()

array([[0., 0., 0., ..., 0., 0., 0.],
       [3., 3., 5., ..., 0., 1., 0.],
       [4., 2., 1., ..., 2., 4., 2.]])

In [69]:
X_stack.round()

array([[ 1., -1., -1., ...,  0.,  0.,  1.],
       [ 3.,  2.,  6., ...,  0.,  1.,  0.],
       [ 4.,  1.,  2., ...,  2.,  3.,  2.]])

In [70]:
dagX.round(1)

array([[0. , 0. , 0.6],
       [0. , 0. , 0. ],
       [0. , 1.1, 0. ]])

In [71]:
W.round(1)

array([[ 0. , -0. ,  0. ],
       [ 0. ,  0. ,  0. ],
       [ 0.2,  0.2,  0. ]])